# Stacking Age and without Age


In [4]:
import pandas as pd

df = pd.read_csv('../data/raw/train.csv')
dfX = df.drop('Survived', axis=1)
dfy = df.Survived

In [5]:
import sys
sys.path.append("../")
from titansurv.pipeline import pipeline1 as dp1
from titansurv.utils import print_params

../titansurv/pipeline/data_pipeline.py:41: UserWarning: Please set the data first using set_data method!
  warnings.warn('Please set the data first using set_data method!')


## Pipeline 1 Age with Mean Imputation

In [6]:
dp1.get_description(markdown=True)

The following were the preprocessing steps used: 
1. **Embarked**: Dropped NA rows and applied OneHotEncoding
2. **Age** : Applied Mean Imputation and Mean Normalization
3. **Fare**: Mean Normalization
4. **Sex**: OneHotEncoding
5. **Name**: Categorised into ['Mr', 'Mrs', 'Miss', 'Master', 'Special']<br/> 
    5.1 Rename [Mlle, Ms] -> Miss      
    5.2 Rename [Mme] -> Mrs     
    5.3 Put the Rest -> Special     
    Then performed OneHotEncoding
6. **Ticket** categorized into [1: numeric, 0: else] <br/>
    6.1 Remove special characters but not space <br/>
    6.2 Replace numeric strings by 'numeric' <br/>
    6.3 Split on space and keep the first item <br/>
 Then applied binarizer for [1: numeric, 0: else]
7. **SibSp** binned into [0, 1, >1] and applied OneHotEncoding
8. **Parch** binned into [0, 1, >1] and applied OneHotEncoding

Tuned ML model: **RandomForestClassifier** using GridSearchCV

## Pipeline 2 Drop Age column

In [7]:
preprocess_data = dp1.preprocess_data

In [8]:
print_params(preprocess_data)

['n_jobs',
 'remainder',
 'sparse_threshold',
 'transformer_weights',
 'transformers',
 'verbose',
 'enc',
 'imp_scaler',
 'scaler',
 'pre_Name',
 'pre_Cabin',
 'pre_Ticket',
 'Pre_SibSp',
 'Pre_Parch',
 'enc__categories',
 'enc__drop',
 'enc__dtype',
 'enc__handle_unknown',
 'enc__sparse',
 'imp_scaler__memory',
 'imp_scaler__steps',
 'imp_scaler__verbose',
 'imp_scaler__imp',
 'imp_scaler__scaler',
 'imp_scaler__imp__add_indicator',
 'imp_scaler__imp__copy',
 'imp_scaler__imp__fill_value',
 'imp_scaler__imp__missing_values',
 'imp_scaler__imp__strategy',
 'imp_scaler__imp__verbose',
 'imp_scaler__scaler__copy',
 'imp_scaler__scaler__with_mean',
 'imp_scaler__scaler__with_std',
 'scaler__copy',
 'scaler__with_mean',
 'scaler__with_std',
 'pre_Name__memory',
 'pre_Name__steps',
 'pre_Name__verbose',
 'pre_Name__featurize',
 'pre_Name__enc',
 'pre_Name__featurize__accept_sparse',
 'pre_Name__featurize__check_inverse',
 'pre_Name__featurize__func',
 'pre_Name__featurize__inv_kw_args',
 '

In [9]:
from titansurv.preprocessing import modify_transformer_est
from sklearn.base import clone

In [10]:
dp2 = clone(dp1)
dp2

DataPipeline

In [11]:
dp2.preprocess_data = modify_transformer_est(preprocess_data, impute_scaler='drop')

## Now Combine the predictions from the two using a StackingClassifier

In [13]:
dp1.data = None
dp2.data = None

In [14]:
dp1.ycol

'Survived'

In [17]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
stack_pipe = StackingClassifier([dp1], meta_classifier=LogisticRegression())